In [1]:
import cv2

In [2]:
import face_recognition

ModuleNotFoundError: No module named 'face_recognition'

In [3]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=e705c0bc1b1eb2c86b8a09fec2594273ae4b4ffd9b91081af1d9832c185eff32
  Stored in directory: /root/.cache/pip/wheels/8f/47/c8/f44c5aebb7507f7c8a2c0bd23151d732d0f0bd6884ad4ac635
Successfully built face-recognition-models


In [4]:
import numpy as np

In [23]:
import os
import cv2
import face_recognition
import numpy as np

def load_known_faces():
    known_encodings = []
    known_names = []

    # Ensure the 'known_faces' directory exists before trying to list its contents
    if not os.path.exists("known_faces"):
        print("Error: 'known_faces' directory not found. Please create it and add face images.")
        return [], []

    face_files = [f for f in os.listdir("known_faces") if f.endswith(".jpg") or f.endswith(".png")]
    if not face_files:
        print("Warning: No face images found in 'known_faces' directory.")

    for filename in face_files:
        image_path = os.path.join("known_faces", filename)
        print(f"Loading face image: {image_path}")
        image = face_recognition.load_image_file(image_path)
        # Get the 128-d face encoding
        face_encodings_in_image = face_recognition.face_encodings(image)
        if face_encodings_in_image:
            encoding = face_encodings_in_image[0]
            known_encodings.append(encoding)
            known_names.append(os.path.splitext(filename)[0])
        else:
            print(f"No face found in {filename}. Skipping.")

    return known_encodings, known_names

def run_recognition(video_source=None): # Modified to accept video_source
    known_encodings, known_names = load_known_faces()

    if not known_encodings:
        print("No known faces loaded. Exiting recognition.")
        return

    if video_source is None:
        print("Error: No video source provided. Please call run_recognition() with a path to a video file, e.g., run_recognition('my_video.mp4')")
        return

    video_capture = cv2.VideoCapture(video_source) # Use video_source here

    if not video_capture.isOpened():
        print(f"Error: Could not open video source '{video_source}'. Please ensure the file exists and is a valid video format.")
        return

    print(f"Starting face recognition from video source: {video_source}")
    while True:
        ret, frame = video_capture.read()

        if not ret:
            print("Reached end of video or failed to read frame.")
            break

        # Resize frame for faster processing (optional)
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Detect faces and get embeddings
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Check for matches
            matches = face_recognition.compare_faces(known_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            if matches: # Only proceed if there are actual matches to compare distances
                face_distances = face_recognition.face_distance(known_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_names[best_match_index]

            # Scale back up face locations since the frame we detected in was 1/4 size
            top *= 4; right *= 4; bottom *= 4; left *= 4

            # Draw Box
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            # Draw Label
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)

        # In Colab, cv2.imshow will not work directly. We need to display the frame differently.
        # For now, we will comment out the display part and focus on processing.
        # If you want to see the output in Colab, you would need to save frames or use specific Colab display utilities.
        # For example, using IPython.display.Image or google.colab.patches.cv2_imshow for a single frame or a sequence.

        # Example for displaying a single frame in a loop (may be slow):
        # from IPython.display import Image, display
        # _, buffer = cv2.imencode('.jpg', frame)
        # display(Image(data=buffer.tobytes()))
        # If you are processing a full video, displaying each frame might not be practical.

    video_capture.release()
    cv2.destroyAllWindows()

# Removed the if __name__ == "__main__": block from this cell.
# You will now explicitly call run_recognition() from another cell.

In [28]:
from google.colab.patches import cv2_imshow # Use this for Google Colab

def recognize_in_image(image_path):
    # 1. Load known faces (Same logic as before)
    known_encodings = []
    known_names = []

    if not os.path.exists("known_faces"):
        print("Folder 'known_faces' not found!")
        return

    for filename in os.listdir("known_faces"):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            img = face_recognition.load_image_file(f"known_faces/{filename}")
            encoding = face_recognition.face_encodings(img)[0]
            known_encodings.append(encoding)
            known_names.append(os.path.splitext(filename)[0])

    # 2. Load the image you want to test
    test_image = face_recognition.load_image_file(image_path)

    # 3. Detect faces and get their encodings
    face_locations = face_recognition.face_locations(test_image)
    face_encodings = face_recognition.face_encodings(test_image, face_locations)

    # Convert to BGR for OpenCV drawing
    output_image = cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR)

    # 4. Loop through each face found in the image
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_encodings, face_encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        if len(face_distances) > 0:
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_names[best_match_index]

        # 5. Draw the results
        cv2.rectangle(output_image, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(output_image, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # 6. Show the final image
    cv2_imshow(output_image)

# Usage
recognize_in_image('test_photo.jpg')

FileNotFoundError: [Errno 2] No such file or directory: 'test_photo.jpg'

In [29]:
import os

if os.path.exists('known_faces'):
    files_in_known_faces = os.listdir('known_faces')
    if files_in_known_faces:
        print("Contents of 'known_faces' directory:")
        for f in files_in_known_faces:
            print(f)
    else:
        print("'known_faces' directory is empty.")
else:
    print("Directory 'known_faces' does not exist.")

'known_faces' directory is empty.


# Task
Upload the `test_photo.jpg` file to the Colab environment.

## Upload Test Image

### Subtask:
Upload the image file you want to analyze (e.g., 'test_photo.jpg') to your Colab environment.


## Upload Test Image

### Subtask:
Upload the image file you want to analyze (e.g., 'test_photo.jpg') to your Colab environment.

#### Instructions
1. In the left sidebar of Google Colab, click on the folder icon to open the 'Files' tab.
2. Click on the 'Upload to session storage' icon (it looks like a file with an arrow pointing upwards).
3. Select the `test_photo.jpg` file from your local machine and upload it to the Colab environment.

Once you have uploaded the `test_photo.jpg` file, you can proceed with the next steps.

## Ensure 'known_faces' Directory Exists and Contains Images

### Subtask:
Make sure the 'known_faces' directory exists and contains images of faces you want to recognize, as the recognition function relies on it.


## Ensure 'known_faces' Directory Exists and Contains Images

### Subtask:
Make sure the 'known_faces' directory exists and contains images of faces you want to recognize, as the recognition function relies on it.

#### Instructions
1.  **Check if a directory named `known_faces` exists** in your Colab environment. You can do this by navigating the 'Files' tab in the left sidebar.
2.  If the `known_faces` directory **does not exist, create it**. You can create a new folder directly in the 'Files' tab.
3.  If the directory is empty or does not contain the images you wish to recognize, **upload the relevant face images** (e.g., `.jpg`, `.png` files) into the `known_faces` directory. Make sure each image contains a single face clearly visible.
4.  **Rename the uploaded face image files** to correspond to the name of the person in the image (e.g., `John_Doe.jpg`). The file name (without extension) will be used as the recognized name.

**Once you have completed these steps, please confirm by responding 'Done'.**

## Ensure 'known_faces' Directory Exists and Contains Images

### Subtask:
Make sure the 'known_faces' directory exists and contains images of faces you want to recognize, as the recognition function relies on it.

#### Instructions
1.  **Check if a directory named `known_faces` exists** in your Colab environment. You can do this by navigating the 'Files' tab in the left sidebar.
2.  If the `known_faces` directory **does not exist, create it**. You can create a new folder directly in the 'Files' tab.
3.  If the directory is empty or does not contain the images you wish to recognize, **upload the relevant face images** (e.g., `.jpg`, `.png` files) into the `known_faces` directory. Make sure each image contains a single face clearly visible.
4.  **Rename the uploaded face image files** to correspond to the name of the person in the image (e.g., `John_Doe.jpg`). The file name (without extension) will be used as the recognized name.

**Once you have completed these steps, please confirm by responding 'Done'.**

## Execute Image Recognition

### Subtask:
Once 'test_photo.jpg' is uploaded and 'known_faces' is ready, run the cell again. The `recognize_in_image('test_photo.jpg')` call should now be able to find and process the image.


## Summary:

### Q&A
The task was to upload the `test_photo.jpg` file to the Colab environment. The solution involved providing the user with step-by-step instructions to manually upload the file using the Colab 'Files' tab. Additionally, instructions were provided to ensure a `known_faces` directory exists, contains relevant images, and these images are appropriately named for face recognition.

### Data Analysis Key Findings
*   The primary method for completing the subtasks was by generating markdown instructions for manual user action within the Colab environment.
*   For uploading `test_photo.jpg`, the agent generated instructions detailing how to use the 'Files' tab and 'Upload to session storage' feature.
*   For ensuring the `known_faces` directory, the agent provided instructions on how to check for, create, upload images to, and rename images within the directory.
*   The agent successfully delivered clear, numbered instructions for both manual setup steps, indicating its role was to guide the user rather than execute file operations directly.

### Insights or Next Steps
*   The approach effectively guides users through necessary manual setup steps for file management and directory organization in the Colab environment.
*   For future tasks involving file uploads or directory creation, consider exploring Colab's API capabilities for programmatic execution to reduce manual user intervention, if feasible.
